In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

In [2]:
games = pd.concat([pd.read_csv('../data/steam_games_cleaned_1.csv'), pd.read_csv('../data/steam_games_cleaned_2.csv'), pd.read_csv('../data/steam_games_cleaned_3.csv')], axis=0).drop('Unnamed: 0', axis=1)
print(games.shape)
games.head()

(56778, 473)


,appid,name,developer,publisher,positive,negative,userscore,owners,average_forever,average_2weeks,...,Web Publishing,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,10,Counter-Strike,Valve,Valve,184296,4785,0,"10,000,000 .. 20,000,000",11666,1107,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.217837
1,20,Team Fortress Classic,Valve,Valve,5232,873,0,"2,000,000 .. 5,000,000",91,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,30,Day of Defeat,Valve,Valve,4881,542,0,"5,000,000 .. 10,000,000",403,0,...,0.0,0.0,0.0,0.0,0.0,0.015287,0.313376,0.0,0.0,0.000000
3,40,Deathmatch Classic,Valve,Valve,1791,402,0,"5,000,000 .. 10,000,000",33,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,12486,633,0,"5,000,000 .. 10,000,000",322,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000


In [3]:
games.set_index('name', inplace=True)

In [4]:
games['has_tag'] = [games.iloc[i,42:].values.max() for i in range(len(games))]
games['has_tag'].value_counts()

1.0    48315
0.0     8463
Name: has_tag, dtype: int64

In [8]:
%%time
# index = 20000
# index_title = games[games['has_tag'] == 1].iloc[index-1:index].index[0]
recommender = games[games['has_tag'] == 1].loc[ : , 'Action' : 'e-sports']
sparse_rec = sparse.csr_matrix(recommender)
dists = pairwise_distances(sparse_rec, metric='cosine')
#simils = cosine_similarity(sparse_rec)
recommender_df = pd.DataFrame(dists, columns=recommender.index, index=recommender.index)

Wall time: 4min 44s


In [9]:
recommender_df.shape

(48315, 48315)

In [10]:
search = 'Beat Hazard'
titles = recommender[recommender.index.str.contains(search)].index
print(titles)

Index(['Beat Hazard', 'Beat Hazard 2'], dtype='object', name='name')


In [20]:
top_recommendations = recommender_df[search].sort_values()[1:21]
top_recs_df = pd.DataFrame(top_recommendations)
# top_recs_df['Genre'] = [games['genre'].loc[game] for game in top_recs_df.index]
# top_recs_df['Avg. Players (2 weeks)'] = [games['average_2weeks'].loc[game] for game in top_recs_df.index]
# top_recs_df['userscore'] = [games['userscore'].loc[game] for game in top_recs_df.index]
top_recs_df[top_recs_df[search] < 0.25]

,Beat Hazard
name,
Symphony,0.099637
Beat Hazard 2,0.133285
Bullet Beat: Musical Shoot'em up,0.192314
AudioSurf,0.206729
Audiosurf 2,0.227873
The Polynomial - Space of the music,0.231299


In [36]:
dict(recommender_df[search].sort_values()[1:11])

{'Symphony': 0.09963674017408575,
 'Beat Hazard 2': 0.13328485536173917,
 "Bullet Beat: Musical Shoot'em up": 0.19231447034971083,
 'AudioSurf': 0.20672910454569982,
 'Audiosurf 2': 0.22787255420119845,
 'The Polynomial - Space of the music': 0.2312993194709675,
 'DubWars': 0.25984660024257267,
 'Intralism': 0.28438729242185223,
 'KickBeat Steam Edition': 0.2908600216739957,
 'Frederic: Resurrection of Music': 0.29632407309994135}

In [38]:
top_20_dicts = []
    for game_index in recommender_df.index:
        top_20_dicts.append({game_index : dict(recommender_df[game_index].sort_values()[1:21])})

IndentationError: unexpected indent (<ipython-input-38-347c5484a286>, line 2)